In [1]:
from random import shuffle, randint
import pandas as pd

In [2]:
def create_new_shoe(n=6):
    shoe = [1,2,3,4,5,6,7,8,9,10,10,10,10]*4*n
    shuffle(shoe)
    return shoe, randint(60,80)

def deal(shoe, hand):
    card = shoe.pop()
    hand.append(card)

def deal_players_hands(shoe, players):
    for _ in range(2):
        for hand in players:
            deal(shoe, hand)

def clear_players_hands(players):
    for hand in players:
        while len(hand) > 0:
            hand.pop()
            
def play_dealer(shoe, dealer_hand):
    if 1 in dealer_hand:
        dealer_hand.remove(1); dealer_hand.append(11)
        
    while sum(dealer_hand) < 17:
        deal(shoe, dealer_hand)
        
        if sum(dealer_hand) > 21 and 11 in dealer_hand:
            dealer_hand.remove(11); dealer_hand.append(1)

def play_player_basic_strategy(shoe, dealer_hand, player_hand):
    dealer_visible_card = dealer_hand[0]
    
    if dealer_visible_card >= 7 or dealer_visible_card == 1:
        if 1 in player_hand:
            player_hand.remove(1); player_hand.append(11)
        while sum(player_hand) < 17:
            deal(shoe, player_hand)

            if sum(player_hand) > 21 and 11 in player_hand:
                player_hand.remove(11); player_hand.append(1)
    else:
        'do nothing'
        
def record_starting_hands(r, dealer_hand, player_hand):
    r['player_start_sum'].append(sum(player_hand))
    r['player_start_hand'].append(tuple(sorted(player_hand)))
    r['dealer_start_sum'].append(sum(dealer_hand))
    r['dealer_visible_card'].append(dealer_hand[0])
    r['dealer_start_hand'].append(tuple(sorted(dealer_hand)))

def record_ending_hands(r, dealer_hand, player_hand):
    if 11 in dealer_hand: dealer_hand.remove(11); dealer_hand.append(1)
    if 11 in player_hand: player_hand.remove(11); player_hand.append(1)
    r['dealer_end_hand'].append(tuple(sorted(dealer_hand)))
    r['dealer_end_sum'].append(sum(dealer_hand))
    if sum(dealer_hand) > 21: r['dealer_busted'].append(True)
    else: r['dealer_busted'].append(False)
    r['player_end_hand'].append(tuple(sorted(player_hand)))
    r['player_end_sum'].append(sum(player_hand))
    if sum(player_hand) > 21: r['player_busted'].append(True)
    else: r['player_busted'].append(False)
    
def score_game(r, dealer_hand, player_hand):
    player_sum, dealer_sum = sum(player_hand), sum(dealer_hand)
    if player_sum > 21:
        r['winner'].append('Dealer'); r['winner_number'].append(-1)
    elif player_sum == 21:
        r['winner'].append('Player'); r['winner_number'].append(1.5)
    elif dealer_sum > 21:
        r['winner'].append('Player'); r['winner_number'].append(1)
    elif dealer_sum > player_sum:
        r['winner'].append('Dealer'); r['winner_number'].append(-1)
    elif dealer_sum < player_sum:
        r['winner'].append('Player'); r['winner_number'].append(1)
    else:
        r['winner'].append('Push'); r['winner_number'].append(0)

def new_record():
    r = {'dealer_start_hand': [], 'dealer_start_sum': [], 
         'dealer_end_hand': [], 'dealer_end_sum': [], 
         'dealer_busted': [], 'dealer_visible_card': [],
         'player_start_hand': [], 'player_start_sum': [], 
         'player_end_hand': [], 'player_end_sum': [], 
         'player_busted': [], 'winner': [], 'winner_number': []}
    return r

## Game Simulation (1,000,000 Hands):

In [3]:
# initialize game/records variables
r = new_record()
records = pd.DataFrame(r)
dealer_hand, player_hand, record = [], [], {}

In [4]:
# create new shoe
shoe, new_shoe_trigger = create_new_shoe(8)
number_of_rounds, round_count = 1000000, 0
r = new_record()

# play round
while round_count < number_of_rounds:
    # deal hands to player and dealer
    deal_players_hands(shoe, [dealer_hand, player_hand])

    # record starting hands for player and dealer
    record_starting_hands(r, dealer_hand, player_hand)

    # play basic strategy for player
    play_player_basic_strategy(shoe, dealer_hand, player_hand)

    # only play dealer if player has not busted
    if sum(player_hand) <= 21: play_dealer(shoe, dealer_hand)

    # score game
    score_game(r, dealer_hand, player_hand)

    # record ending hands
    record_ending_hands(r, dealer_hand, player_hand)

    # if end shoe is near, shuffle and make new shoe
    if len(shoe) < new_shoe_trigger:
        shoe, new_shoe_trigger = create_new_shoe(8)

    # clear player hands and record
    clear_players_hands([dealer_hand, player_hand])
    round_count += 1
    
# store records
new_records = pd.DataFrame(r)
records = records.append(new_records, ignore_index=True)

In [5]:
print('Returns Percent:', str(round(records['winner_number'].sum()/records['winner_number'].count()*100,2))+'%')

Returns Percent: -6.61%
